# first, generate a barcode read from read 2, 3, 4

basically followed this: https://github.com/alexdobin/STAR/issues/825

In [ ]:
module load mawk
for file in fastq_indrop/*_1.fastq.gz
do
    echo $file
    sample=`basename ${file%_1.fastq.gz}`
    echo $sample
    SAMPLE_R2=fastq_indrop/${sample}_2.fastq.gz
    SAMPLE_R4=fastq_indrop/${sample}_4.fastq.gz
    SAMPLE_R24=fastq_indrop/${sample}_24.fastq.gz
    echo $SAMPLE_R24
    paste <(zcat ${SAMPLE_R2}) <(zcat ${SAMPLE_R4}) | mawk '{if (NR%4==1) {print $1 " 2:4:paste"} else if (NR%4==3) {print $1} else {print $1 $2}}' | gzip > ${SAMPLE_R24} &
done

In [17]:
jobs

In [ ]:
paste \ 
    <(zcat ${SAMPLE_R2}) \
    <(zcat ${SAMPLE_R3}) \
    <(zcat ${SAMPLE_R4}) \
    | mawk '{if ((NR%4==1) && ($1 == $3) && ($3 == $5)) {print $1 " 2:3:4:paste"} else if ((NR%4==1) && ($1 != $3)) {print "singleton"} else if ((NR%4==1) && ($3 != $5)) {print "singleton"} else if (NR%4==3) {print $1} else {print $1 $2 $3}}' \
    | gzip > ${SAMPLE_R234.fastq.gz}

# then, generate bc list

In [ ]:
module load mawk
cartesian(){
awk -F '\t' \
    -v "bc1_list=$1" \
    -v "bc2_list=$2" '
    {
        if ( FILENAME == bc1_list ) {
            BC1[$0];
        }
        else if ( FILENAME == bc2_list ) {
             BC2[$0];
        }
    } END {
        for (bc1 in BC1) {
           for (bc2 in BC2) {
               print bc1 bc2
           }
        }
     } ' $1 $2 > $3
}


In [ ]:
cat 20220127_indrop_barcodes_kalish_384x384/gel_barcode2_list.txt | tr 'ACGT' 'TGCA' | rev > 20220127_indrop_barcodes_kalish_384x384/gel_barcode2_list_revcomp.txt

In [ ]:
cartesian 20220127_indrop_barcodes_kalish_384x384/gel_barcode2_list.txt 20220127_indrop_barcodes_kalish_384x384/gel_barcode2_list_revcomp.txt 20220127_indrop_barcodes_kalish_384x384/full_whitelist

# then map

pos 1-8: bc1
pos 9-16: 8 bp index
pos 17-24: bc2 (reverse complement of whitelist)  
pos 25-30: UMI

In [1]:
star_solo_indrop_rna_mapping_unique_cellfilter () {
    local fastq_R1_filename="${1}";
    local fastq_R2_filename="${2}";
    local bam_filename="${3}";
    # 3 barcode lists:
    local whitelist_part1_filename='indrop_whitelist/gel_barcode2_list.txt';
    local whitelist_part2_filename='indrop_whitelist/index_whitelist.txt';
    local whitelist_part3_filename='indrop_whitelist/gel_barcode2_list_revcomp.txt';
    star_reference_dir='/lustre1/project/stg_00002/lcb/fderop/data/00000000_genomes/mm10_STAR_2.7.8_premrna';
    sjdbgtf='/lustre1/project/stg_00002/lcb/fderop/data/00000000_genomes/mm10_STAR_2.7.8_premrna/genes.gtf'
    #module load STAR/2.7.5b-foss-2018a;
    #module load SAMtools/1.10-foss-2018a;
    # Map with STAR solo
    /lustre1/project/stg_00002/lcb/fderop/scripts/STAR-2.7.8a/bin/Linux_x86_64/STAR \
        --runThreadN 32 \
        --runMode alignReads \
        --outSAMtype BAM SortedByCoordinate \
        --sysShell /bin/bash \
        --genomeDir "${star_reference_dir}" \
        --readFilesIn "${fastq_R1_filename}" "${fastq_R2_filename}" \
        --readFilesCommand 'gzip -c -d' \
        --soloCBwhitelist "${whitelist_part1_filename}" "${whitelist_part2_filename}" "${whitelist_part3_filename}" \
        --soloType CB_UMI_Complex \
        --soloCBposition 0_0_0_7 0_8_0_15 0_16_0_23 \
        --soloUMIposition 0_24_0_29 \
        --sjdbGTFfile $sjdbgtf \
        --soloCellFilter CellRanger2.2 2000 0.99 10 \
        --soloCBmatchWLtype 1MM \
        --outFilterMultimapNmax 1 \
        --outSAMattributes NH HI AS nM CB UB CR CY UR UY \
        --outFileNamePrefix ${bam_filename%bam} \
        --outReadsUnmapped Fastx \
        --quantMode GeneCounts \
        --bamRemoveDuplicatesType UniqueIdentical \
        --soloFeatures Gene GeneFull
        # Index BAM file.
    # samtools index "${bam_filename%bam}Aligned.sortedByCoord.out.bam"
}

Now we run this star solo script in a for loop on all the R2 fastq files in `fastq/`

In [2]:
dir=demultiplexed_indrop_index
mkdir $dir
for sample in fastq_indrop/*_1.fastq.gz
do
    NAME=${sample%_1.fastq.gz}
    OUT=${NAME##*/}
    echo $NAME
    star_solo_indrop_rna_mapping_unique_cellfilter \
        ${NAME}_1.fastq.gz \
        ${NAME}_234.fastq.gz \
        $dir/$OUT.bam
done

mkdir: cannot create directory ‘demultiplexed_indrop_index’: File exists
fastq_indrop/SRR10545068
Jan 31 17:55:49 ..... started STAR run
Jan 31 17:55:51 ..... loading genome
Jan 31 17:56:51 ..... processing annotations GTF
Jan 31 17:56:58 ..... started mapping
Jan 31 18:11:36 ..... finished mapping
Jan 31 18:11:40 ..... started Solo counting
Jan 31 18:13:05 ..... finished Solo counting
Jan 31 18:13:05 ..... started sorting BAM
Jan 31 18:14:17 ..... finished successfully
fastq_indrop/SRR10545069
Jan 31 18:14:34 ..... started STAR run
Jan 31 18:14:35 ..... loading genome
Jan 31 18:14:51 ..... processing annotations GTF
Jan 31 18:14:53 ..... started mapping
Jan 31 18:27:46 ..... finished mapping
Jan 31 18:27:50 ..... started Solo counting
Jan 31 18:28:46 ..... finished Solo counting
Jan 31 18:28:46 ..... started sorting BAM
Jan 31 18:29:40 ..... finished successfully
fastq_indrop/SRR10545070
Jan 31 18:29:53 ..... started STAR run
Jan 31 18:29:56 ..... loading genome
Jan 31 18:30:11 ..... 

actually, mapping one big fastq of all samples will be better. Then, umi demultiplexing will work properly.